# Example: Solving the Primal and Dual Production Allocation Problem
Consider a hypothetical manufacturing facility that converts a cheap raw material into five possible products $\left\{p_{i}\right\}_{i=1}^{i=5}$ using processes $\left\{P_{j}\right\}_{j=1}^{j=4}$. The capacity of each process, i.e., the maximum number of product units that can be produced per process, the unit price of each product, and the capacity required to manufacture product $i$ using process $j$  are given by:

| Process | Capacity | p$_{1}$ |  p$_{2}$ | p$_{3}$ | p$_{4}$ | p$_{5}$ |
| :---: | :---: | --- | --- | --- | --- | --- |
P$_{1}$ | 160 | 1.2 | 1.3 | 0.7 | 0.0 | 0.5
P$_{2}$ | 200 | 0.7 | 2.2 | 1.6 | 0.5 | 1.0 |
P$_{3}$ | 120 | 0.9 | 0.7 | 1.3 | 1.0 | 0.8 |
P$_{4}$ | 280 | 1.4 | 2.8 | 0.5 | 1.2 | 0.6 |
Unit price $ | -- | 18 | 25 | 10 | 12 | 15

Estimate the optimum production quantities for the products $p_{i}$ that maximize the total revenue. Let's try to solve this problem using linear programming.

## Setup
This example requires several external libraries and a function to compute the outer product. Let's download and install these packages and call our `Include.jl` file.

In [3]:
include("Include.jl");

## Task 1: Setup linear programming problem
In this task, we setup the linear programing problem. First, we specify the number of products that we are producing, and the number of processes that we have to produce 

In [5]:
number_of_products = 5; # How many products do we have?
number_of_processes = 4; # How many processes do we have?

Next, define the system constraint matrix $\mathbf{A}$. The entries of $a_{ij}\in\mathbf{A}$ represent the capacity cost of manufacturing product $j$ using process $i$.

In [7]:
A = [
    1.2 1.3 0.7 0.0 0.5 ; # process 1
    0.7 2.2 1.6 0.5 1.0 ; # process 2
    0.9 0.7 1.3 1.0 0.8 ; # process 3
    1.4 2.8 0.5 1.2 0.6 ; # process 4
];

Then, we'll define the right-hand side vector $\mathbf{b}$ (these represent the maximum capacities of the machines).

In [9]:
b = [160, 200, 120, 280]; # Setup the right hand side vector (capcity)

The last thing we need to specify is the coefficients of the objective function $\mathbf{c}$. The entry $c_{i}\in\mathbf{c}$ is the unit price of product $i$ in our set of products.

In [11]:
c = [18.0,25.0,10.0,12.0,15.0]; # revenue for each product

Finally, each of our decision variables $x_{i}$ (how much of product $i$ to manufacture) is bounded such that $L_{i}\leq{x}_{i}\leq{U_{i}}$, where $L_{i}$ denotes the lower bound (in this case all variables must be greater than zero) and $U_{i}$ denotes the upper bound (in this case, some big number).

In [13]:
bounds = [
    0.0 1000.0; # L U compound 1
    0.0 1000.0; # L U compound 2
    0.0 1000.0; # L U compound 3
    0.0 1000.0; # L U compound 4
    0.0 1000.0; # L U compound 5
];

Now that we have all the components, create an instance of [the `MyLinearProgrammingProblemModel` type](src/Types.jl) using [a `build(...)` method](src/Factory.jl). We input problem parameters, such as the objective function coefficients, cost values, total budget values, an initial guess, and allocation bounds.

In [15]:
primal_problem = build(MyLinearProgrammingProblemModel, (
 
    c = c, # coefficients in the objective function (unit costs)
    A = A, # production table (processes to compounds)
    b = b, # capacity constraints
    
    # how much of x₁ and x₂ can be we buy?
    lb = bounds[:,1], # lower bound
    ub = bounds[:,2] # uppber bound
));

In [16]:
primal_problem.A

4×5 Matrix{Float64}:
 1.2  1.3  0.7  0.0  0.5
 0.7  2.2  1.6  0.5  1.0
 0.9  0.7  1.3  1.0  0.8
 1.4  2.8  0.5  1.2  0.6

## Task 2: Solve the primal linear programming problem
In this task, we solve the production problem as a [Linear programming problem](https://en.wikipedia.org/wiki/Linear_programming). Let's not worry so much about how a linear programming solver works (we are invoking the `buy` side of `buy` versus `build`). Instead, let's try to understand how the problem is structured, and then we'll dig into the details of the solver later. 
* To solve the primal problem, we use [the `solve(...)` method](src/Solver.jl) with the `primal_problem::MyLinearProgrammingProblemModel` variable and store the solution in the `primal_solution` variable.
* __What's up with the try-catch?__ The call to [the `solve(...)` method](src/Solve.jl) has a check to see if the solution returned by the solver is feasible. This check uses [the @assert macro](), which can throw [an AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError). So, we wrap our call to [the `solve(...)` method](src/Solve.jl) in [try-catch block](https://docs.julialang.org/en/v1/manual/control-flow/#The-try/catch-statement) to catch this exception gracefully should it occur.

In [18]:
primal_solution = nothing;
try
    primal_solution = solve(primal_problem)
catch error
    println(error)
end

Dict{String, Any} with 2 entries:
  "argmax"          => [0.0, 0.0, 0.0, 0.0, 133.333]
  "objective_value" => 2000.0

In [19]:
optimal_revenue = primal_solution["objective_value"];
println("The optimal revenue is: $(optimal_revenue) USD")

The optimal revenue is: 2000.0000000000002 USD


### Visualize product mix
`Unhide` the code block below to see how we make a table holding the optimal production mixture produced by the primal problem.
* __Summary__: Given the default parameters, the optimal product mix does not include product `3.` Product `2` was the most manufactured product, followed by `1`, then `5` and finally `4`.

In [21]:
let
    df = DataFrame();
    C = primal_solution["argmax"]
    for i ∈ 1:number_of_products 
        row_df = (
            product = i,
            production = C[i],
        )
        push!(df, row_df);
    end
    pretty_table(df, tf=tf_simple)
end

========== =============
  product   production 
    Int64      Float64 
========== =============
        1          0.0
        2          0.0
        3          0.0
        4          0.0
        5      133.333
========== =============


### Visualize machine capacity
`Unhide` the code block below to see how we make a table holding which machines were utilized to produce our optimal product mixture.
* __Summary__: Even though we have inequality constraints, all machines were allocated to capacity when maximizing the revenue for selling the products. 

In [23]:
let
    df = DataFrame();
    C = primal_solution["argmax"]
    P = A*C;
    
    for i ∈ 1:number_of_processes
        row_df = (
            process = i,
            utilization = P[i],
            capacity = b[i]
        )
        push!(df, row_df);
    end
    pretty_table(df, tf=tf_simple)
end

========== ============= ===========
  process   utilization   capacity 
    Int64       Float64      Int64 
========== ============= ===========
        1       66.6667        160
        2       133.333        200
        3       106.667        120
        4          80.0         80
========== ============= ===========


## Task 3: Solve the dual production planning problem.
In this task, we'll solve the dual production planning problem. To better understand the dual, let's look at the units. The dual problem is given by:
$$
\begin{eqnarray*}
\text{minimize}~\mathcal{O}^{\prime}(\mathbf{y}) &=& \sum_{i=1}^{m} b_{i}\cdot{y}_{i}\\
\text{subject to}~\mathbf{A}^{T}\cdot\mathbf{y} &\geq&\mathbf{c}\\
\text{and}~y_{i}&\geq&{0}\qquad{i=1,2,\dots,m}
\end{eqnarray*}
$$
where $b_{i}$ denotes the maximum machine capacities from the primal problem (units: `capacity`), and $\mathbf{c}$ is the unit cost vector (units: `USD/unit`). The entries in the constraint matrix $a_{ij}\in\mathbf{A}$ have units of `capacity/unit.` Thus, the dual variables $y_{i}\in\mathbf{y}$ have units of `USD/capacity.` Putting all this together:
* The dual problem is minimizing the cost of operating the facility so that we can generate at least the revenue of the primal problem. Thus, we are exploring the problem through the lens of efficiency.

First, we'll create the appropriate problem type, and then we'll pass this problem type to the solver. Let's begin by creating [a `MyLinearProgrammingDualProblemModel` instance](src/Types.jl) using [a `build(...)` method](src/Factory.jl). We'll save the dual problem model in the `dual_problem::MyLinearProgrammingDualProblemModel` variable:

In [26]:
dual_problem = let

        bounds = zeros(number_of_processes,2);
        bounds[:,2] .= 1000; # what is going on with the .= ??
    
        problem = build(MyLinearProgrammingDualProblemModel, (

            c = b, # dual: the coefficients in the objective function are primal right-hand sides
            b = c, # dual: the right-hand side of the constraints is now the coeff of the obj function from the primal problem
            A = transpose(A), # constraint matrix is the transpose(A)

            lb = bounds[:,1], # lower bound (zeros)
            ub = bounds[:,2]  # upper bound (some big number)
        ));

    problem
end;

Now that we have the dual problem model, we can solve the problem [using the `solve(...)` method](src/Solver.jl).
* To solve the primal problem, we pass the `dual_problem::MyLinearProgrammingDualProblemModel` problem instance to [the `solve(...)` method](src/Solver.jl). We store the solution in the `dual_solution` variable.
* __What's up with the try-catch?__ The call to [the `solve(...)` method](src/Solve.jl) has a check to see if the solution returned by the solver is feasible. This check uses [the @assert macro](), which can throw [an AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError). So, we wrap our call to [the `solve(...)` method](src/Solve.jl) in [try-catch block](https://docs.julialang.org/en/v1/manual/control-flow/#The-try/catch-statement) to catch this exception gracefully should it occur.

In [28]:
dual_solution = nothing;
try
    dual_solution = solve(dual_problem)
catch error
    println(error);
end

Dict{String, Any} with 2 entries:
  "argmin"          => [0.0, 0.0, 0.0, 25.0]
  "objective_value" => 2000.0